In [129]:
import pandas as pd

In [130]:
df = pd.read_csv('/content/Daily_Public_Transport_Passenger_Journeys.csv')

In [131]:
df

,Date,Local Route,Light Rail,Peak Service,Rapid Route,School,Other
0,30-08-2024,16436,10705,225,19026,3925,59.0
1,15-09-2023,15499,10671,267,18421,4519,61.0
2,28-12-2021,1756,2352,0,3775,0,13.0
3,11-01-2023,10536,8347,223,14072,0,48.0
4,11-09-2021,820,612,0,1283,0,11.0
...,...,...,...,...,...,...,...
1913,19-01-2023,11337,8671,262,14881,0,66.0
1914,16-08-2023,16791,11800,387,20936,4684,113.0
1915,10-01-2022,5732,3548,73,6644,0,25.0
1916,04-05-2020,3717,1952,70,4411,131,19.0


In [132]:
null_counts = df.isnull().sum()
print("Count of null values in each column:\n", null_counts[null_counts > 0])

Count of null values in each column:
 Other    20
dtype: int64


In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [134]:
data = pd.read_csv('/content/Daily_Public_Transport_Passenger_Journeys.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True, errors='coerce')
print(data.head())

        Date  Local Route  Light Rail  Peak Service  Rapid Route  School  \
0 2024-08-30        16436       10705           225        19026    3925   
1 2023-09-15        15499       10671           267        18421    4519   
2 2021-12-28         1756        2352             0         3775       0   
3 2023-01-11        10536        8347           223        14072       0   
4 2021-09-11          820         612             0         1283       0   

   Other  
0   59.0  
1   61.0  
2   13.0  
3   48.0  
4   11.0  


In [135]:
data['Other'] = data['Other'].fillna(data['Other'].median())


In [136]:
print(data['Other'].isnull().sum())

0


In [137]:
data['Total Passengers'] = data[['Local Route', 'Light Rail', 'Peak Service',
                                 'Rapid Route', 'School', 'Other']].sum(axis=1)

data['Month'] = data['Date'].dt.month
data['Day_of_Week'] = data['Date'].dt.dayofweek
data['Is_Weekend'] = (data['Day_of_Week'] >= 5).astype(int)

print(data[['Total Passengers', 'Month', 'Day_of_Week', 'Is_Weekend']].head())  # Verify new features


   Total Passengers  Month  Day_of_Week  Is_Weekend
0           50376.0      8            4           0
1           49438.0      9            4           0
2            7896.0     12            1           0
3           33226.0      1            2           0
4            2726.0      9            5           1


In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [141]:
features = ['Month', 'Day_of_Week', 'Is_Weekend']
X = data[features]
routes = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School', 'Other']
y = data[routes]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [152]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2

model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(6)
])

In [153]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_data=(X_test_scaled, y_test), callbacks=[early_stop])


Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 69337048.0000 - mae: 5368.2935 - val_loss: 65775764.0000 - val_mae: 5272.7100
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68058184.0000 - mae: 5308.8589 - val_loss: 65767964.0000 - val_mae: 5272.2822
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69347224.0000 - mae: 5401.4121 - val_loss: 65747568.0000 - val_mae: 5271.0840
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 68810512.0000 - mae: 5372.4839 - val_loss: 65697772.0000 - val_mae: 5268.1050
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71062232.0000 - mae: 5466.8975 - val_loss: 65598796.0000 - val_mae: 5262.1704
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 68511544.0000 - mae: 5334.3115 - val_loss: 65448628.0000 - val_mae: 5253.2251
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70843472.0000 - mae: 5425.9355 - val_loss: 65251636.0000 - val_mae: 5241.4839
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0

In [154]:
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8385217.0000 - mae: 1683.5083  
Test Loss: 8527351.0000, Test MAE: 1722.7480


In [155]:
routes = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School', 'Other']

def preprocess_input(user_input):
    input_data = pd.DataFrame([user_input], columns=['Month', 'Day_of_Week', 'Is_Weekend'])
    scaled_input = scaler.transform(input_data)
    return scaled_input

def predict_passenger_counts(user_input):
    processed_input = preprocess_input(user_input)
    predicted_values = model.predict(processed_input)
    print(f"Predicted Values Shape: {predicted_values.shape}")

    predictions = dict(zip(routes, predicted_values[0]))
    return predictions

user_input = {
    'Month': 11,
    'Day_of_Week': 1,
    'Is_Weekend': 0
}

predicted_passengers = predict_passenger_counts(user_input)

print("\nPredicted Total Passengers for Each Route:")
for route, passengers in predicted_passengers.items():
    print(f"{route}: {passengers:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted Values Shape: (1, 6)

Predicted Total Passengers for Each Route:
Local Route: 12971.32
Light Rail: 8564.75
Peak Service: 249.72
Rapid Route: 15444.16
School: 3334.63
Other: 61.73
